In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import cv2
import tensorflow 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import SimpleITK as sitk
import shutil
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam


Normalização e tratamento do dataset

In [2]:
diretorio = 'C:/Users/User/Documents/ADNI/Img_ADNI/'

arquivos = os.listdir(diretorio)

rotulos = {}
# incluir os rótulos e converter em número
for arquivo in arquivos:
    if arquivo.startswith('NC'):
        rotulos[arquivo] = 0  # Rótulo 1 para NC
    elif arquivo.startswith('MCI'):
        rotulos[arquivo] = 1  # Rótulo 2 para MCI
    elif arquivo.startswith('DA'):
        rotulos[arquivo] = 2  # Rótulo 3 para DA

imagens = []
labels = []

for arquivo in arquivos:
    caminho = os.path.join(diretorio, arquivo)
    imagem = cv2.imread(caminho) 
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)  # Converte para RGB
    imagem = cv2.resize(imagem, (128, 128))
    imagens.append(imagem)
    labels.append(rotulos[arquivo])

imagens = np.array(imagens)
labels = np.array(labels)



In [4]:

imagens = imagens / 255.0
imagens = np.expand_dims(imagens, axis=-1)  
labels = to_categorical(labels, num_classes=3)

# Dividir os dados treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(imagens, labels, test_size=0.2, random_state=42)

# Exibir algumas informações sobre os dados
print(f'Número total de imagens: {len(imagens)}')
print(f'Dimensões das imagens: {imagens.shape[1:]}')
print(f'Número de classes: {labels.shape[1]}')

NameError: name 'to_categorical' is not defined

Modelo CNN

In [30]:
def build_cnn_model(input_shape):
    input_layer = Input(shape=input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)
    flatten = Flatten()(pool2)
    dense1 = Dense(128, activation='relu')(flatten)
    
    output_nc = Dense(3, activation='softmax', name='output_nc')(dense1)
    output_mci = Dense(3, activation='softmax', name='output_mci')(dense1)
    output_da = Dense(3, activation='softmax', name='output_da')(dense1)
    
    model = Model(inputs=input_layer, outputs=[output_nc, output_mci, output_da])
    return model

model_cnn = build_cnn_model((128, 128, 3))

model_cnn.compile(optimizer='adam',
                  loss={'output_nc': 'categorical_crossentropy',
                        'output_mci': 'categorical_crossentropy',
                        'output_da': 'categorical_crossentropy'},
                  metrics={'output_nc': 'accuracy',
                           'output_mci': 'accuracy',
                           'output_da': 'accuracy'})

history_cnn = model_cnn.fit(X_train, {'output_nc': y_train, 'output_mci': y_train, 'output_da': y_train},
                            validation_data=(X_test, {'output_nc': y_test, 'output_mci': y_test, 'output_da': y_test}),
                            epochs=10, batch_size=32)



Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 148ms/step - loss: 3.4698 - output_da_accuracy: 0.3082 - output_mci_accuracy: 0.3236 - output_nc_accuracy: 0.3337 - val_loss: 3.2975 - val_output_da_accuracy: 0.3270 - val_output_mci_accuracy: 0.3238 - val_output_nc_accuracy: 0.3206
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 138ms/step - loss: 3.2905 - output_da_accuracy: 0.3727 - output_mci_accuracy: 0.3685 - output_nc_accuracy: 0.3639 - val_loss: 3.2940 - val_output_da_accuracy: 0.3619 - val_output_mci_accuracy: 0.3429 - val_output_nc_accuracy: 0.3587
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 137ms/step - loss: 3.2719 - output_da_accuracy: 0.4056 - output_mci_accuracy: 0.3871 - output_nc_accuracy: 0.3913 - val_loss: 3.3105 - val_output_da_accuracy: 0.3429 - val_output_mci_accuracy: 0.3587 - val_output_nc_accuracy: 0.3333
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 140ms/step - loss: 3.2059 - output_da_accuracy: 0.4439 - output_mci_accuracy: 0.4036 - output_nc_accuracy: 0.4279 - val_loss: 3.3554 - va

Modelo Densenet

In [33]:
def build_densenet_model(input_shape):
    input_layer = Input(shape=input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)
    flatten = Flatten()(pool2)
    dense1 = Dense(128, activation='relu')(flatten)
    
    output_nc = Dense(3, activation='softmax', name='output_nc_densenet')(dense1)
    output_mci = Dense(3, activation='softmax', name='output_mci_densenet')(dense1)
    output_da = Dense(3, activation='softmax', name='output_da_densenet')(dense1)
    
    model = Model(inputs=input_layer, outputs=[output_nc, output_mci, output_da])
    return model

model_densenet = build_densenet_model((128, 128, 1))  # Input shape (128, 128, 1) para imagens em escala de cinza

model_densenet.compile(optimizer='adam',
                       loss={'output_nc_densenet': 'categorical_crossentropy',
                             'output_mci_densenet': 'categorical_crossentropy',
                             'output_da_densenet': 'categorical_crossentropy'},
                       metrics={'output_nc_densenet': 'accuracy',
                                'output_mci_densenet': 'accuracy',
                                'output_da_densenet': 'accuracy'})

history_densenet = model_densenet.fit(X_train, {'output_nc_densenet': y_train, 'output_mci_densenet': y_train, 'output_da_densenet': y_train},
                                      validation_data=(X_test, {'output_nc_densenet': y_test, 'output_mci_densenet': y_test, 'output_da_densenet': y_test}),
                                      epochs=10, batch_size=32)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - loss: 15.6310 - output_da_densenet_accuracy: 0.3610 - output_mci_densenet_accuracy: 0.3755 - output_nc_densenet_accuracy: 0.4143 - val_loss: 54.7232 - val_output_da_densenet_accuracy: 0.4500 - val_output_mci_densenet_accuracy: 0.4500 - val_output_nc_densenet_accuracy: 0.4500
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - loss: 58.4201 - output_da_densenet_accuracy: 0.5035 - output_mci_densenet_accuracy: 0.3728 - output_nc_densenet_accuracy: 0.2722 - val_loss: 100.4456 - val_output_da_densenet_accuracy: 0.1000 - val_output_mci_densenet_accuracy: 0.4500 - val_output_nc_densenet_accuracy: 0.4500
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - loss: 89.9998 - output_da_densenet_accuracy: 0.1244 - output_mci_densenet_accuracy: 0.4293 - output_nc_densenet_accuracy: 0.5066 - val_loss: 162.4260 - val_output_da_densenet_accuracy: 0.4500 - val_output_mci_densenet_accuracy: 0.1000 - val_output_nc_densenet_accuracy: 0.1000
Epoch 4/10
4/4

Combinação dos modelos

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D

input_layer_combined = Input(shape=(128, 128, 3))

output_cnn = model_cnn(input_layer_combined)
output_densenet = model_densenet(input_layer_combined)

combined_output_nc = Dense(3, activation='softmax', name='combined_output_nc')(
    Average()([output_cnn[0], output_densenet[0]])) 
combined_output_mci = Dense(3, activation='softmax', name='combined_output_mci')(
    Average()([output_cnn[1], output_densenet[1]]))
combined_output_da = Dense(3, activation='softmax', name='combined_output_da')(
    Average()([output_cnn[2], output_densenet[2]]))


model_combined = Model(inputs=input_layer_combined,
                       outputs=[combined_output_nc, combined_output_mci, combined_output_da])


model_combined.compile(optimizer='adam',
                       loss={'combined_output_nc': 'categorical_crossentropy',
                             'combined_output_mci': 'categorical_crossentropy',
                             'combined_output_da': 'categorical_crossentropy'},
                       metrics={'combined_output_nc': 'accuracy',
                                'combined_output_mci': 'accuracy',
                                'combined_output_da': 'accuracy'})




NameError: name 'Average' is not defined

In [71]:
history_cnn = model_cnn.fit(X_train, {'output_nc': y_train, 'output_mci': y_train, 'output_da': y_train},
                            validation_data=(X_test, {'output_nc': y_test, 'output_mci': y_test, 'output_da': y_test}),
                            epochs=10, batch_size=32)

history_densenet = model_densenet.fit(X_train, {'output_nc_densenet': y_train, 'output_mci_densenet': y_train, 'output_da_densenet': y_train},
                                      validation_data=(X_test, {'output_nc_densenet': y_test, 'output_mci_densenet': y_test, 'output_da_densenet': y_test}),
                                      epochs=10, batch_size=32)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 12165.0029 - output_da_accuracy: 0.5137 - output_mci_accuracy: 0.5095 - output_nc_accuracy: 0.2537 - val_loss: 10446.4404 - val_output_da_accuracy: 0.2000 - val_output_mci_accuracy: 0.4000 - val_output_nc_accuracy: 0.2000
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - loss: 16764.6250 - output_da_accuracy: 0.2081 - output_mci_accuracy: 0.2545 - output_nc_accuracy: 0.2041 - val_loss: 14189.1768 - val_output_da_accuracy: 0.4000 - val_output_mci_accuracy: 0.4000 - val_output_nc_accuracy: 0.4000
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 22777.6094 - output_da_accuracy: 0.2283 - output_mci_accuracy: 0.4815 - output_nc_accuracy: 0.2161 - val_loss: 17994.1367 - val_output_da_accuracy: 0.2000 - val_output_mci_accuracy: 0.4000 - val_output_nc_accuracy: 0.2000
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - loss: 27501.5879 - output_da_accuracy: 0.3374 - output_mci_accuracy: 0.4494 - output_nc_accuracy: 0.2304 - v

In [72]:
from tensorflow.keras.layers import Average


input_layer_combined = Input(shape=(128, 128, 1)) 


output_cnn = model_cnn(input_layer_combined)
output_densenet = model_densenet(input_layer_combined)


combined_output_nc = Dense(3, activation='softmax', name='combined_output_nc')(
    Average()([output_cnn[0], output_densenet[0]]))
combined_output_mci = Dense(3, activation='softmax', name='combined_output_mci')(
    Average()([output_cnn[1], output_densenet[1]]))
combined_output_da = Dense(3, activation='softmax', name='combined_output_da')(
    Average()([output_cnn[2], output_densenet[2]]))


model_combined = Model(inputs=input_layer_combined,
                       outputs=[combined_output_nc, combined_output_mci, combined_output_da])


model_combined.compile(optimizer='adam',
                       loss={'combined_output_nc': 'categorical_crossentropy',
                             'combined_output_mci': 'categorical_crossentropy',
                             'combined_output_da': 'categorical_crossentropy'},
                       metrics={'combined_output_nc': 'accuracy',
                                'combined_output_mci': 'accuracy',
                                'combined_output_da': 'accuracy'})



Model: "functional_36"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_24       │ [(None, 3),       │  7,392,905 │ input_layer_22[0… │
│ (Functional)        │ (None, 3), (None, │            │                   │
│                     │ 3)]               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_28       │ [(None, 3),       │  7,392,905 │ input_layer_22[0… │
│ (Functional)        │ (None, 3), (None, │            │                   │
│                     │ 3)]               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_9 (Average) │ (None, 3)         │          0 │ functional_24[3]… │
│                     │                   │            │ functional_28[3]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_10          │ (None, 3)         │          0 │ functional_24[3]… │
│ (Average)           │                   │            │ functional_28[3]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_11          │ (None, 3)         │          0 │ functional_24[3]… │
│ (Average)           │                   │            │ functional_28[3]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combined_output_nc  │ (None, 3)         │         12 │ average_9[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combined_output_mci │ (None, 3)         │         12 │ average_10[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combined_output_da  │ (None, 3)         │         12 │ average_11[0][0]  │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,785,846 (56.40 MB)

 Trainable params: 14,785,846 (56.40 MB)

 Non-trainable params: 0 (0.00 B)

Treinamento combinação modelos

In [74]:
history_combined = model_combined.fit(X_train,
                                     {'combined_output_nc': y_train, 'combined_output_mci': y_train, 'combined_output_da': y_train},
                                     validation_data=(X_test,
                                                      {'combined_output_nc': y_test, 'combined_output_mci': y_test, 'combined_output_da': y_test}),
                                     epochs=10, batch_size=32)


results = model_combined.evaluate(X_test,
                                  {'combined_output_nc': y_test, 'combined_output_mci': y_test, 'combined_output_da': y_test})


loss = results[0]
acc_nc = results[1]
acc_mci = results[2]
acc_da = results[3]

print(f"Loss: {loss}")
print(f"Acurácia NC: {acc_nc}")
print(f"Acurácia MCI: {acc_mci}")
print(f"Acurácia DA: {acc_da}")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step - combined_output_da_accuracy: 0.2910 - combined_output_mci_accuracy: 0.2910 - combined_output_nc_accuracy: 0.1894 - loss: 0.0000e+00 - output_da_accuracy: 0.0000e+00 - output_da_densenet_accuracy: 0.0000e+00 - output_mci_accuracy: 0.0000e+00 - output_mci_densenet_accuracy: 0.0000e+00 - output_nc_accuracy: 0.0000e+00 - output_nc_densenet_accuracy: 0.0000e+00 - val_combined_output_da_accuracy: 0.4000 - val_combined_output_mci_accuracy: 0.4000 - val_combined_output_nc_accuracy: 0.2000 - val_loss: 0.0000e+00 - val_output_da_accuracy: 0.0000e+00 - val_output_da_densenet_accuracy: 0.0000e+00 - val_output_mci_accuracy: 0.0000e+00 - val_output_mci_densenet_accuracy: 0.0000e+00 - val_output_nc_accuracy: 0.0000e+00 - val_output_nc_densenet_accuracy: 0.0000e+00
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step - combined_output_da_accuracy: 0.2806 - combined_output_mci_accuracy: 0.2806 - combined_output_nc_accuracy: 0.1915 - loss: 0.0000e+00 - ou

In [88]:
import numpy as np


predictions_train = model_combined.predict(X_train)


std_deviation = np.std(predictions_train)

print(f"Desvio Padrão das Previsões: {std_deviation}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Desvio Padrão das Previsões: 0.10892697423696518


In [89]:

def jaccard_similarity_coefficient(y_true, y_pred):
    intersection = np.sum(np.minimum(y_true, y_pred))
    union = np.sum(np.maximum(y_true, y_pred))
    return intersection / union


predictions_train_classes = np.argmax(predictions_train, axis=-1)
y_train_classes = np.argmax(y_train, axis=-1)


jaccard_similarity = jaccard_similarity_coefficient(y_train_classes, predictions_train_classes)

print(f"Coeficiente de Similaridade de Jaccard: {jaccard_similarity}")

Coeficiente de Similaridade de Jaccard: 0.3795454545454545


In [122]:
from sklearn.metrics import roc_auc_score
import numpy as np


predictions_train_array = np.array(predictions_train)


y_train_classes = np.argmax(y_train, axis=1)


auc_nc = roc_auc_score((y_train_classes == 0).astype(int), predictions_train_array[:, 0])
auc_mci = roc_auc_score((y_train_classes == 1).astype(int), predictions_train_array[:, 1])
auc_da = roc_auc_score((y_train_classes == 2).astype(int), predictions_train_array[:, 2])

print(f"AUC NC: {auc_nc}")
print(f"AUC MCI: {auc_mci}")
print(f"AUC DA: {auc_da}")



AUC NC: 1.0
AUC MCI: 1.0
AUC DA: 1.0


In [123]:
import numpy as np


def calcular_desvio_padrao(data):
    return np.std(data, axis=0)


std_train = calcular_desvio_padrao(X_train)
std_validation = calcular_desvio_padrao(X_test)

print(f"Desvio Padrão (Treinamento): {std_train}")
print(f"Desvio Padrão (Validação): {std_validation}")

Desvio Padrão (Treinamento): [[[0.27979591]
  [0.2858786 ]
  [0.29391499]
  ...
  [0.31257587]
  [0.29436152]
  [0.27536017]]

 [[0.31586358]
  [0.26298826]
  [0.3071377 ]
  ...
  [0.279102  ]
  [0.28797988]
  [0.29725758]]

 [[0.28829517]
  [0.28508321]
  [0.31047972]
  ...
  [0.2748418 ]
  [0.27770476]
  [0.25721112]]

 ...

 [[0.29286919]
  [0.28221415]
  [0.28058384]
  ...
  [0.27787163]
  [0.28502711]
  [0.29434321]]

 [[0.2679376 ]
  [0.28446832]
  [0.26371736]
  ...
  [0.29414459]
  [0.29345042]
  [0.28671864]]

 [[0.30582797]
  [0.29610616]
  [0.28863542]
  ...
  [0.29296269]
  [0.25209707]
  [0.26577789]]]
Desvio Padrão (Validação): [[[0.26862894]
  [0.28498587]
  [0.26051279]
  ...
  [0.31193762]
  [0.32997624]
  [0.26513169]]

 [[0.2508592 ]
  [0.27847648]
  [0.23717125]
  ...
  [0.24683851]
  [0.27262915]
  [0.28507644]]

 [[0.32662891]
  [0.27735096]
  [0.2923065 ]
  ...
  [0.29353099]
  [0.30016033]
  [0.27885538]]

 ...

 [[0.26993608]
  [0.27719233]
  [0.25257343]
  ...

In [124]:
from sklearn.metrics.pairwise import cosine_similarity


def calcular_coeficiente_similaridade(data1, data2):
    similarity = cosine_similarity(data1, data2)
    return similarity.mean()


similarity_train_validation = calcular_coeficiente_similaridade(X_train.reshape(len(X_train), -1), X_test.reshape(len(X_test), -1))

print(f"Coeficiente de Similaridade de Dados (Treinamento vs. Validação): {similarity_train_validation}")

Coeficiente de Similaridade de Dados (Treinamento vs. Validação): 0.7500643896507188


In [145]:
from sklearn.metrics import roc_auc_score
import numpy as np

y_true_train = {
    'combined_output_nc': np.array([0, 1, 1, 0]), 
    'combined_output_mci': np.array([1, 0, 1, 0]), 
    'combined_output_da': np.array([0, 1, 0, 1])
}

y_pred_train = {
    'combined_output_nc': np.array([0.2, 0.8, 0.4, 0.6]),
    'combined_output_mci': np.array([0.3, 0.6, 0.7, 0.9]),
    'combined_output_da': np.array([0.1, 0.4, 0.5, 0.3]) 
}

# Exemplo de dados de treinamento
y_true_train = {
    'combined_output_nc': np.array([0, 1, 1, 0]),
    'combined_output_mci': np.array([1, 0, 1, 0]), 
    'combined_output_da': np.array([0, 1, 0, 1]) 
}

y_pred_train = {
    'combined_output_nc': np.array([0.2, 0.8, 0.4, 0.6]),  
    'combined_output_mci': np.array([0.3, 0.6, 0.7, 0.9]),
    'combined_output_da': np.array([0.1, 0.4, 0.5, 0.3]) 
}



auc_train_nc = roc_auc_score(y_true_train['combined_output_nc'], y_pred_train['combined_output_nc'])
auc_train_mci = roc_auc_score(y_true_train['combined_output_mci'], y_pred_train['combined_output_mci'])
auc_train_da = roc_auc_score(y_true_train['combined_output_da'], y_pred_train['combined_output_da'])

print(f"AUC (Treinamento - NC): {auc_train_nc}")
print(f"AUC (Treinamento - MCI): {auc_train_mci}")
print(f"AUC (Treinamento - DA): {auc_train_da}")




AUC (Treinamento - NC): 0.75
AUC (Treinamento - MCI): 0.25
AUC (Treinamento - DA): 0.5
